## Построение модели логистической регрессии - Урок 4

![Инфографика: Логистическая vs. линейная регрессия](../../../../../../translated_images/linear-vs-logistic.ba180bf95e7ee66721ba10ebf2dac2666acbd64a88b003c83928712433a13c7d.ru.png)

#### **[Тест перед лекцией](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/15/)**

#### Введение

В этом заключительном уроке по регрессии, одной из базовых *классических* техник машинного обучения, мы рассмотрим логистическую регрессию. Этот метод используется для выявления закономерностей и предсказания бинарных категорий. Это конфета с шоколадом или без? Это заболевание заразное или нет? Выберет ли клиент этот продукт или нет?

В этом уроке вы узнаете:

-   Методы логистической регрессии

✅ Углубите свои знания о работе с этим типом регрессии в этом [модуле обучения](https://learn.microsoft.com/training/modules/introduction-classification-models/?WT.mc_id=academic-77952-leestott)

## Предварительные знания

Работая с данными о тыквах, мы уже достаточно с ними знакомы, чтобы понять, что есть одна бинарная категория, с которой мы можем работать: `Color`.

Давайте построим модель логистической регрессии, чтобы предсказать, *какого цвета, скорее всего, будет данная тыква* (оранжевая 🎃 или белая 👻).

> Почему мы говорим о бинарной классификации в уроке, посвященном регрессии? Только для удобства терминологии, так как логистическая регрессия [на самом деле является методом классификации](https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression), хотя и основанным на линейных принципах. Узнайте о других способах классификации данных в следующей группе уроков.

Для этого урока нам понадобятся следующие пакеты:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) — это [набор пакетов для R](https://www.tidyverse.org/packages), разработанных для того, чтобы сделать работу с данными быстрее, проще и увлекательнее!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) — это [фреймворк пакетов](https://www.tidymodels.org/packages/) для моделирования и машинного обучения.

-   `janitor`: [Пакет janitor](https://github.com/sfirke/janitor) предоставляет простые инструменты для анализа и очистки "грязных" данных.

-   `ggbeeswarm`: [Пакет ggbeeswarm](https://github.com/eclarke/ggbeeswarm) позволяет создавать диаграммы в стиле "роя пчел" с использованием ggplot2.

Вы можете установить их с помощью команды:

`install.packages(c("tidyverse", "tidymodels", "janitor", "ggbeeswarm"))`

Или же используйте следующий скрипт, который проверяет наличие необходимых пакетов и устанавливает их, если они отсутствуют.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, janitor, ggbeeswarm)


## **Определение вопроса**

Для наших целей мы будем выражать это как бинарный выбор: «Белый» или «Не белый». В нашем наборе данных также есть категория «полосатый», но случаев с ней очень мало, поэтому мы не будем ее использовать. Она исчезает, как только мы удаляем пустые значения из набора данных.

> 🎃 Забавный факт: иногда мы называем белые тыквы «призрачными» тыквами. Их сложно вырезать, поэтому они не так популярны, как оранжевые, но выглядят они круто! Таким образом, мы могли бы переформулировать наш вопрос как: «Призрачная» или «Не призрачная». 👻

## **О логистической регрессии**

Логистическая регрессия отличается от линейной регрессии, о которой вы узнали ранее, несколькими важными аспектами.

#### **Бинарная классификация**

Логистическая регрессия не предлагает тех же возможностей, что и линейная регрессия. Первая предоставляет прогноз о `бинарной категории` («оранжевая или не оранжевая»), тогда как вторая способна предсказывать `непрерывные значения`, например, исходя из происхождения тыквы и времени сбора урожая, *насколько вырастет ее цена*.

![Инфографика от Dasani Madipalli](../../../../../../translated_images/pumpkin-classifier.562771f104ad5436b87d1c67bca02a42a17841133556559325c0a0e348e5b774.ru.png)

### Другие классификации

Существуют и другие типы логистической регрессии, включая мультиномиальную и порядковую:

- **Мультиномиальная**, которая включает более одной категории — «Оранжевая, Белая и Полосатая».

- **Порядковая**, которая включает упорядоченные категории, полезные, если мы хотим логически упорядочить наши результаты, например, тыквы, которые упорядочены по конечному числу размеров (мини, маленький, средний, большой, очень большой, гигантский).

![Мультиномиальная vs порядковая регрессия](../../../../../../translated_images/multinomial-vs-ordinal.36701b4850e37d86c9dd49f7bef93a2f94dbdb8fe03443eb68f0542f97f28f29.ru.png)

#### **Переменные НЕ обязаны коррелировать**

Помните, как линейная регрессия работала лучше с более коррелированными переменными? Логистическая регрессия — противоположность: переменные не обязаны совпадать. Это подходит для наших данных, которые имеют довольно слабую корреляцию.

#### **Нужно много чистых данных**

Логистическая регрессия даст более точные результаты, если использовать больше данных; наш небольшой набор данных не является оптимальным для этой задачи, так что имейте это в виду.

✅ Подумайте о типах данных, которые хорошо подходят для логистической регрессии.

## Упражнение — приведение данных в порядок

Сначала немного очистите данные, удалив пустые значения и выбрав только некоторые столбцы:

1. Добавьте следующий код:


In [ ]:
# Load the core tidyverse packages
library(tidyverse)

# Import the data and clean column names
pumpkins <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/2-Regression/data/US-pumpkins.csv") %>% 
  clean_names()

# Select desired columns
pumpkins_select <- pumpkins %>% 
  select(c(city_name, package, variety, origin, item_size, color)) 

# Drop rows containing missing values and encode color as factor (category)
pumpkins_select <- pumpkins_select %>% 
  drop_na() %>% 
  mutate(color = factor(color))

# View the first few rows
pumpkins_select %>% 
  slice_head(n = 5)


Вы всегда можете взглянуть на ваш новый датафрейм, используя функцию [*glimpse()*](https://pillar.r-lib.org/reference/glimpse.html), как показано ниже:


In [ ]:
pumpkins_select %>% 
  glimpse()


Давайте подтвердим, что мы действительно будем решать задачу бинарной классификации:


In [ ]:
# Subset distinct observations in outcome column
pumpkins_select %>% 
  distinct(color)


### Визуализация - категориальный график
На данный момент вы снова загрузили данные о тыквах и очистили их, чтобы сохранить набор данных, содержащий несколько переменных, включая Color. Давайте визуализируем этот датафрейм в ноутбуке, используя библиотеку ggplot.

Библиотека ggplot предлагает удобные способы визуализации данных. Например, вы можете сравнить распределения данных для каждого Variety и Color на категориальном графике.

1. Создайте такой график, используя функцию geombar, наши данные о тыквах и указав цветовую схему для каждой категории тыкв (оранжевые или белые):


In [ ]:
# Specify colors for each value of the hue variable
palette <- c(ORANGE = "orange", WHITE = "wheat")

# Create the bar plot
ggplot(pumpkins_select, aes(y = variety, fill = color)) +
  geom_bar(position = "dodge") +
  scale_fill_manual(values = palette) +
  labs(y = "Variety", fill = "Color") +
  theme_minimal()

По данным видно, как информация о Цвете соотносится с Разновидностью.

✅ Учитывая этот категориальный график, какие интересные исследования вы можете представить?


### Предобработка данных: кодирование признаков

Наш набор данных о тыквах содержит строковые значения во всех своих столбцах. Работа с категориальными данными интуитивно понятна для людей, но не для машин. Алгоритмы машинного обучения лучше работают с числами. Именно поэтому кодирование является очень важным этапом на этапе предобработки данных, так как оно позволяет преобразовать категориальные данные в числовые, не теряя при этом информации. Хорошее кодирование способствует созданию качественной модели.

Для кодирования признаков существуют два основных типа кодировщиков:

1. Ordinal encoder: хорошо подходит для порядковых переменных, то есть категориальных переменных, данные которых имеют логическую последовательность, как, например, столбец `item_size` в нашем наборе данных. Он создает отображение, где каждая категория представлена числом, соответствующим порядку категории в столбце.

2. Categorical encoder: хорошо подходит для номинальных переменных, то есть категориальных переменных, данные которых не имеют логической последовательности, как, например, все признаки, кроме `item_size`, в нашем наборе данных. Это кодирование с использованием метода "one-hot", что означает, что каждая категория представлена бинарным столбцом: закодированная переменная равна 1, если тыква принадлежит данной разновидности, и 0 в противном случае.

Tidymodels предоставляет еще один удобный пакет: [recipes](https://recipes.tidymodels.org/) — пакет для предобработки данных. Мы определим `recipe`, который указывает, что все столбцы предикторов должны быть закодированы в набор целых чисел, затем `prep`, чтобы оценить необходимые величины и статистики для любых операций, и, наконец, `bake`, чтобы применить вычисления к новым данным.

> Обычно recipes используется как инструмент предобработки для моделирования, где он определяет, какие шаги должны быть применены к набору данных, чтобы подготовить его к моделированию. В этом случае **настоятельно рекомендуется** использовать `workflow()` вместо ручной оценки рецепта с помощью prep и bake. Мы рассмотрим это чуть позже.
>
> Однако сейчас мы используем recipes + prep + bake, чтобы указать, какие шаги должны быть применены к набору данных для его подготовки к анализу данных, а затем извлечь предобработанные данные с примененными шагами.


In [ ]:
# Preprocess and extract data to allow some data analysis
baked_pumpkins <- recipe(color ~ ., data = pumpkins_select) %>%
  # Define ordering for item_size column
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  # Convert factors to numbers using the order defined above (Ordinal encoding)
  step_integer(item_size, zero_based = F) %>%
  # Encode all other predictors using one hot encoding
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE) %>%
  prep(data = pumpkin_select) %>%
  bake(new_data = NULL)

# Display the first few rows of preprocessed data
baked_pumpkins %>% 
  slice_head(n = 5)


✅ Каковы преимущества использования порядкового кодировщика для столбца Item Size?

### Анализ взаимосвязей между переменными

Теперь, когда мы предварительно обработали данные, мы можем проанализировать взаимосвязи между признаками и меткой, чтобы понять, насколько хорошо модель сможет предсказывать метку на основе признаков. Лучший способ провести такой анализ — это визуализация данных.  
Мы снова будем использовать функцию ggplot geom_boxplot_, чтобы визуализировать взаимосвязи между Item Size, Variety и Color в категориальном графике. Для более наглядного отображения данных мы будем использовать закодированный столбец Item Size и некодированный столбец Variety.


In [ ]:
# Define the color palette
palette <- c(ORANGE = "orange", WHITE = "wheat")

# We need the encoded Item Size column to use it as the x-axis values in the plot
pumpkins_select_plot<-pumpkins_select
pumpkins_select_plot$item_size <- baked_pumpkins$item_size

# Create the grouped box plot
ggplot(pumpkins_select_plot, aes(x = `item_size`, y = color, fill = color)) +
  geom_boxplot() +
  facet_grid(variety ~ ., scales = "free_x") +
  scale_fill_manual(values = palette) +
  labs(x = "Item Size", y = "") +
  theme_minimal() +
  theme(strip.text = element_text(size = 12)) +
  theme(axis.text.x = element_text(size = 10)) +
  theme(axis.title.x = element_text(size = 12)) +
  theme(axis.title.y = element_blank()) +
  theme(legend.position = "bottom") +
  guides(fill = guide_legend(title = "Color")) +
  theme(panel.spacing = unit(0.5, "lines"))+
  theme(strip.text.y = element_text(size = 4, hjust = 0)) 


#### Используйте график роя

Поскольку Color — это бинарная категория (Белый или Не белый), для её визуализации требуется '[специализированный подход](https://github.com/rstudio/cheatsheets/blob/main/data-visualization.pdf)'.

Попробуйте использовать `график роя`, чтобы показать распределение цвета в зависимости от item_size.

Мы будем использовать [пакет ggbeeswarm](https://github.com/eclarke/ggbeeswarm), который предоставляет методы для создания графиков в стиле "роя" с использованием ggplot2. Графики роя позволяют отображать точки, которые обычно накладывались бы друг на друга, так, чтобы они располагались рядом друг с другом.


In [ ]:
# Create beeswarm plots of color and item_size
baked_pumpkins %>% 
  mutate(color = factor(color)) %>% 
  ggplot(mapping = aes(x = color, y = item_size, color = color)) +
  geom_quasirandom() +
  scale_color_brewer(palette = "Dark2", direction = -1) +
  theme(legend.position = "none")


Теперь, когда мы понимаем связь между бинарными категориями цвета и более крупной группой размеров, давайте изучим логистическую регрессию, чтобы определить вероятный цвет тыквы.

## Создайте свою модель

Выберите переменные, которые вы хотите использовать в своей классификационной модели, и разделите данные на обучающую и тестовую выборки. [rsample](https://rsample.tidymodels.org/), пакет из Tidymodels, предоставляет инфраструктуру для эффективного разделения данных и повторного выборочного анализа:


In [ ]:
# Split data into 80% for training and 20% for testing
set.seed(2056)
pumpkins_split <- pumpkins_select %>% 
  initial_split(prop = 0.8)

# Extract the data in each split
pumpkins_train <- training(pumpkins_split)
pumpkins_test <- testing(pumpkins_split)

# Print out the first 5 rows of the training set
pumpkins_train %>% 
  slice_head(n = 5)


🙌 Теперь мы готовы обучить модель, сопоставив обучающие признаки с обучающей меткой (цветом).

Начнем с создания рецепта, который определяет шаги предварительной обработки данных, чтобы подготовить их для моделирования, например: кодирование категориальных переменных в набор целых чисел. Подобно `baked_pumpkins`, мы создаем `pumpkins_recipe`, но не используем `prep` и `bake`, так как это будет включено в рабочий процесс, который вы увидите через несколько шагов.

Существует довольно много способов задать модель логистической регрессии в Tidymodels. См. `?logistic_reg()`. На данный момент мы зададим модель логистической регрессии через стандартный движок `stats::glm()`.


In [ ]:
# Create a recipe that specifies preprocessing steps for modelling
pumpkins_recipe <- recipe(color ~ ., data = pumpkins_train) %>% 
  step_mutate(item_size = ordered(item_size, levels = c('sml', 'med', 'med-lge', 'lge', 'xlge', 'jbo', 'exjbo'))) %>%
  step_integer(item_size, zero_based = F) %>%  
  step_dummy(all_nominal(), -all_outcomes(), one_hot = TRUE)

# Create a logistic model specification
log_reg <- logistic_reg() %>% 
  set_engine("glm") %>% 
  set_mode("classification")


Теперь, когда у нас есть рецепт и спецификация модели, нам нужно найти способ объединить их в один объект, который сначала будет предварительно обрабатывать данные (prep+bake за кулисами), обучать модель на предварительно обработанных данных, а также предоставлять возможность для потенциальных действий по постобработке.

В Tidymodels этот удобный объект называется [`workflow`](https://workflows.tidymodels.org/) и удобно хранит ваши компоненты моделирования.


In [ ]:
# Bundle modelling components in a workflow
log_reg_wf <- workflow() %>% 
  add_recipe(pumpkins_recipe) %>% 
  add_model(log_reg)

# Print out the workflow
log_reg_wf


После того как рабочий процесс *задан*, модель можно `обучить` с помощью функции [`fit()`](https://tidymodels.github.io/parsnip/reference/fit.html). Рабочий процесс оценит рецепт и предварительно обработает данные перед обучением, поэтому нам не придется делать это вручную с использованием prep и bake.


In [ ]:
# Train the model
wf_fit <- log_reg_wf %>% 
  fit(data = pumpkins_train)

# Print the trained workflow
wf_fit


Модель выводит коэффициенты, полученные в процессе обучения.

Теперь, когда мы обучили модель на тренировочных данных, мы можем делать прогнозы на тестовых данных, используя [parsnip::predict()](https://parsnip.tidymodels.org/reference/predict.model_fit.html). Давайте начнем с использования модели для предсказания меток для нашего тестового набора и вероятностей для каждой метки. Если вероятность больше 0.5, предсказанный класс — `WHITE`, в противном случае — `ORANGE`.


In [ ]:
# Make predictions for color and corresponding probabilities
results <- pumpkins_test %>% select(color) %>% 
  bind_cols(wf_fit %>% 
              predict(new_data = pumpkins_test)) %>%
  bind_cols(wf_fit %>%
              predict(new_data = pumpkins_test, type = "prob"))

# Compare predictions
results %>% 
  slice_head(n = 10)


Очень интересно! Это дает больше понимания того, как работает логистическая регрессия.

### Лучшее понимание через матрицу ошибок

Сравнение каждого предсказания с соответствующим "истинным" значением — не самый эффективный способ определить, насколько хорошо модель делает прогнозы. К счастью, у Tidymodels есть еще несколько полезных инструментов: [`yardstick`](https://yardstick.tidymodels.org/) — пакет, который используется для оценки эффективности моделей с помощью метрик производительности.

Одна из метрик производительности, связанная с задачами классификации, — это [`матрица ошибок`](https://wikipedia.org/wiki/Confusion_matrix). Матрица ошибок описывает, насколько хорошо справляется модель классификации. Она показывает, сколько примеров из каждого класса были правильно классифицированы моделью. В нашем случае она покажет, сколько оранжевых тыкв были классифицированы как оранжевые, а сколько белых тыкв — как белые; матрица ошибок также покажет, сколько объектов были отнесены к **неверным** категориям.

Функция [**`conf_mat()`**](https://tidymodels.github.io/yardstick/reference/conf_mat.html) из пакета yardstick вычисляет эту перекрестную таблицу наблюдаемых и предсказанных классов.


In [ ]:
# Confusion matrix for prediction results
conf_mat(data = results, truth = color, estimate = .pred_class)


Давайте разберем матрицу ошибок. Наша модель должна классифицировать тыквы на две бинарные категории: категория `белая` и категория `не-белая`.

-   Если ваша модель предсказывает, что тыква белая, и она действительно относится к категории "белая", это называется `истинно положительное` (true positive), что отображается в верхнем левом углу.

-   Если ваша модель предсказывает, что тыква не белая, но на самом деле она относится к категории "белая", это называется `ложно отрицательное` (false negative), что отображается в нижнем левом углу.

-   Если ваша модель предсказывает, что тыква белая, но на самом деле она относится к категории "не-белая", это называется `ложно положительное` (false positive), что отображается в верхнем правом углу.

-   Если ваша модель предсказывает, что тыква не белая, и она действительно относится к категории "не-белая", это называется `истинно отрицательное` (true negative), что отображается в нижнем правом углу.

| Истина |
|:-----:|

|               |        |       |
|---------------|--------|-------|
| **Предсказано** | БЕЛАЯ | ОРАНЖЕВАЯ |
| БЕЛАЯ         | TP     | FP    |
| ОРАНЖЕВАЯ     | FN     | TN    |

Как вы могли догадаться, желательно иметь больше истинно положительных и истинно отрицательных значений, а также меньше ложно положительных и ложно отрицательных значений, что указывает на лучшее качество работы модели.

Матрица ошибок полезна, так как она позволяет вычислить другие метрики, которые помогают лучше оценить производительность классификационной модели. Давайте рассмотрим некоторые из них:

🎓 Точность (Precision): `TP/(TP + FP)` — доля предсказанных положительных случаев, которые действительно являются положительными. Также называется [положительное прогностическое значение](https://en.wikipedia.org/wiki/Positive_predictive_value "Positive predictive value").

🎓 Полнота (Recall): `TP/(TP + FN)` — доля положительных результатов из общего числа образцов, которые действительно являются положительными. Также известна как `чувствительность`.

🎓 Специфичность (Specificity): `TN/(TN + FP)` — доля отрицательных результатов из общего числа образцов, которые действительно являются отрицательными.

🎓 Точность классификации (Accuracy): `TP + TN/(TP + TN + FP + FN)` — процент меток, предсказанных правильно для выборки.

🎓 F-мера (F Measure): Взвешенное среднее между точностью и полнотой, где лучший результат равен 1, а худший — 0.

Давайте рассчитаем эти метрики!


In [ ]:
# Combine metric functions and calculate them all at once
eval_metrics <- metric_set(ppv, recall, spec, f_meas, accuracy)
eval_metrics(data = results, truth = color, estimate = .pred_class)


## Визуализация ROC-кривой этой модели

Давайте сделаем еще одну визуализацию, чтобы увидеть так называемую [`ROC-кривую`](https://en.wikipedia.org/wiki/Receiver_operating_characteristic):


In [ ]:
# Make a roc_curve
results %>% 
  roc_curve(color, .pred_ORANGE) %>% 
  autoplot()


ROC-кривые часто используются для визуализации работы классификатора в терминах истинных и ложных срабатываний. ROC-кривые обычно отображают `True Positive Rate`/чувствительность по оси Y и `False Positive Rate`/1-специфичность по оси X. Таким образом, важны крутизна кривой и расстояние между диагональной линией и кривой: вам нужна кривая, которая быстро поднимается вверх и уходит за линию. В нашем случае сначала есть ложные срабатывания, а затем линия поднимается вверх и уходит за пределы диагонали.

Наконец, давайте используем `yardstick::roc_auc()`, чтобы вычислить фактическую площадь под кривой (Area Under the Curve). Один из способов интерпретации AUC — это вероятность того, что модель оценит случайный положительный пример выше, чем случайный отрицательный пример.


In [ ]:
# Calculate area under curve
results %>% 
  roc_auc(color, .pred_ORANGE)


Результат составляет около `0.975`. Учитывая, что AUC варьируется от 0 до 1, вы хотите получить высокий показатель, так как модель, которая на 100% точна в своих предсказаниях, будет иметь AUC, равный 1; в данном случае модель *довольно хорошая*.

В будущих уроках по классификации вы узнаете, как улучшить показатели вашей модели (например, как справляться с несбалансированными данными в данном случае).

## 🚀Задание

Логистическая регрессия — это обширная тема! Но лучший способ учиться — это экспериментировать. Найдите набор данных, который подходит для такого типа анализа, и создайте с ним модель. Что вы узнаете? совет: попробуйте [Kaggle](https://www.kaggle.com/search?q=logistic+regression+datasets) для поиска интересных наборов данных.

## Обзор и самостоятельное изучение

Прочитайте первые несколько страниц [этой статьи из Стэнфорда](https://web.stanford.edu/~jurafsky/slp3/5.pdf) о практическом применении логистической регрессии. Подумайте о задачах, которые лучше подходят для одного или другого типа регрессии, которые мы изучали до этого момента. Что будет работать лучше?



---

**Отказ от ответственности**:  
Этот документ был переведен с использованием сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, учитывайте, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникающие в результате использования данного перевода.
